<h1> This notebook is to create make a dataframe of having the list of Toronto's Boroughs with their zipcodes attached to them </h1>

Lets first get the particular page of wikipedia having the said values

In [52]:
import lxml
from lxml import html
import requests
import pandas as pd

Let's first import the beautiful Soup library and the urlLib library

In [53]:
from urllib.request import urlopen

from bs4 import BeautifulSoup as BS

Now we open the url page in an object and then extract the table using findall to an object named table

In [54]:
wikiPage = urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') 

In [55]:
soup = BS(wikiPage,'lxml')

In [56]:
table = soup.find_all('tbody')[0]

now let's create a dataframe to save the values of everything required

In [57]:
dataTable = pd.DataFrame(columns=range(0,3), index = [0])

Let's add the column names now & insert the values

In [58]:
dataTable.columns = ['PostCode','Borough','Neighbourhood']

In [59]:
rowCounter = 0
for row in table.find_all('tr'):
    columnValue = row.find_all('td')
    columncounter =0
    for column in columnValue :
        if(column.get_text() != 'Not assigned'):
            if(columncounter == 0):
                dataTable.set_value(rowCounter,'PostCode',column.get_text())
            elif(columncounter == 1):
                dataTable.set_value(rowCounter,'Borough',column.get_text())
            elif(columncounter == 2):
                dataTable.set_value(rowCounter,'Neighbourhood',column.get_text())
        columncounter = columncounter + 1
    rowCounter = rowCounter + 1
    

C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':
C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


let's see the result

In [60]:
dataTable.head()

PostCode     Borough       Neighbourhood
0      NaN         NaN                 NaN
1      M1A         NaN      Not assigned\n
2      M2A         NaN      Not assigned\n
3      M3A  North York         Parkwoods\n
4      M4A  North York  Victoria Village\n

Now we need to remove the NaN values, not assigned value's rows and the index needs tobe reshuffled accordignly

In [61]:
dataTable.dropna(inplace = True)

In [62]:
dataTable = dataTable[dataTable.Borough != 'Not assigned']


In [63]:
dataTable.reset_index(inplace = True)

In [64]:
dataTable.head()

index PostCode           Borough       Neighbourhood
0      3      M3A        North York         Parkwoods\n
1      4      M4A        North York  Victoria Village\n
2      5      M5A  Downtown Toronto      Harbourfront\n
3      6      M5A  Downtown Toronto       Regent Park\n
4      7      M6A        North York  Lawrence Heights\n

In [65]:
dataTable = dataTable.drop(['index'],axis = 1)

Now we remove the \n from the name fo neighbourhoods, group the common neighbourhoods & boroughs according to the postCodes

In [66]:
dataTable['Neighbourhood'] = dataTable['Neighbourhood'].map(lambda x : x.rstrip('\n'))

In [67]:
dataTable['Borough'] = dataTable[['PostCode','Borough','Neighbourhood']].groupby(['PostCode'])['Borough'].transform(lambda x : ','.join(x))

In [68]:
dataTable['Neighbourhood'] = dataTable[['PostCode','Borough','Neighbourhood']].groupby(['PostCode'])['Neighbourhood'].transform(lambda x : ','.join(x))

In [69]:
dataTable.drop_duplicates(inplace = True)

In [70]:
dataTable.reset_index(inplace = True)

In [71]:
dataTable = dataTable.drop(['index'],axis = 1)

Let's see how it looks now

In [72]:
dataTable.head()

PostCode                            Borough                    Neighbourhood
0      M3A                         North York                        Parkwoods
1      M4A                         North York                 Victoria Village
2      M5A  Downtown Toronto,Downtown Toronto         Harbourfront,Regent Park
3      M6A              North York,North York  Lawrence Heights,Lawrence Manor
4      M7A                       Queen's Park                     Not assigned

Let's import the geocoder module to see the values of Lattitude's & Longitudes

In [76]:
from geopy import geocoders

# USing the geo code data instead of geoCoder as for some reason it's not working

In [77]:
geoData = pd.read_csv('https://cocl.us/Geospatial_data')

In [78]:
geoData.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

Let's rename the column Psotal Codes to PostCode

In [79]:
geoData['PostCode'] = geoData['Postal Code']

In [80]:
geoData = geoData.drop('Postal Code', axis = 1)

Now we merge the geoData table and the dataTable to get the values in one table

In [81]:
dataTable = pd.merge(dataTable,geoData,how = 'inner', on = 'PostCode')

Let's see how it looks now

In [89]:
dataTable.head()

PostCode                            Borough  \
0      M3A                         North York   
1      M4A                         North York   
2      M5A  Downtown Toronto,Downtown Toronto   
3      M6A              North York,North York   
4      M7A                       Queen's Park   

                     Neighbourhood   Latitude  Longitude  
0                        Parkwoods  43.753259 -79.329656  
1                 Victoria Village  43.725882 -79.315572  
2         Harbourfront,Regent Park  43.654260 -79.360636  
3  Lawrence Heights,Lawrence Manor  43.718518 -79.464763  
4                     Not assigned  43.662301 -79.389494

As mentioned earlier , the final cell of the notebook is the shape of the table

In [83]:
dataTable.shape

(103, 5)

In [49]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Let's see how many Boroughs and neighbourhoods are there in our data

In [85]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(dataTable['Borough'].unique()),
        dataTable.shape[0]
    )
)

The dataframe has 32 boroughs and 103 neighborhoods.


Let's get the lat and long of Toronto

In [86]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Let's create the map

In [91]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dataTable['Latitude'], dataTable['Longitude'], dataTable['Borough'], dataTable['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [109]:
toronto_data = pd.DataFrame(columns = ['lat','long','borough','Neighbourhood'],index = [0])

In [116]:
indx = 0
for lat,long,borough,neigh,index in zip(dataTable['Latitude'], dataTable['Longitude'], dataTable['Borough'], dataTable['Neighbourhood'],dataTable.index):
    if('Toronto' in borough):
        toronto_data.set_value(indx,'lat',lat)
        toronto_data.set_value(indx,'long',long)
        toronto_data.set_value(indx,'borough',borough)
        toronto_data.set_value(indx,'Neighbourhood',neigh)
        indx = indx + 1


C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
C:\Users\ezchave\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


In [118]:
toronto_data.head()

lat     long                            borough  \
0  43.6543 -79.3606  Downtown Toronto,Downtown Toronto   
1  43.6572 -79.3789  Downtown Toronto,Downtown Toronto   
2  43.6515 -79.3754                   Downtown Toronto   
3  43.6764  -79.293                       East Toronto   
4  43.6448 -79.3733                   Downtown Toronto   

              Neighbourhood  
0  Harbourfront,Regent Park  
1   Ryerson,Garden District  
2            St. James Town  
3               The Beaches  
4               Berczy Park

Now let's use foresquare for the mapping 

In [119]:
CLIENT_ID = 'HQADI3DUD22CJBEMCEI4MPC4PXS3SVYOJH1DYGOKQIEBPDIQ' # your Foursquare ID
CLIENT_SECRET = 'BJ4DBMDCHV40OW451L4OFEBOLX4FFNJTJHNYIMLZ2J3ZYHYV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HQADI3DUD22CJBEMCEI4MPC4PXS3SVYOJH1DYGOKQIEBPDIQ
CLIENT_SECRET:BJ4DBMDCHV40OW451L4OFEBOLX4FFNJTJHNYIMLZ2J3ZYHYV


In [120]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT = 500
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [122]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['lat'],
                                   longitudes=toronto_data['long']
                                  )
    

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


In [126]:
toronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Adelaide,King,Richmond                                                100   
Berczy Park                                                            57   
Brockton,Exhibition Place,Parkdale Village                             19   
Business Reply Mail Processing Centre 969 Eastern                      34   
CN Tower,Bathurst Quay,Island airport,Harbourfr...                     15   
Cabbagetown,St. James Town                                             44   
Central Bay Street                                                     88   
Chinatown,Grange Park,Kensington Market                               100   
Christie                                                               16   
Church and Wellesley                                                   88   
Commerce Court,Victoria Hotel                                         100   
Davisville                                                             32   
Davisville North                                                        7   
Deer Park,Forest Hill SE,Rathnelly,South Hill,S...                     14   
Design Exchange,Toronto Dominion Centre                               100   
Dovercourt Village,Dufferin                                            20   
First Canadian Place,Underground city                                 100   
Forest Hill North,Forest Hill West                                      4   
Harbord,University of Toronto                                          34   
Harbourfront East,Toronto Islands,Union Station                       100   
Harbourfront,Regent Park                                               46   
High Park,The Junction South                                           23   
Lawrence Park                                                           4   
Little Portugal,Trinity                                                63   
Moore Park,Summerhill East                                              3   
North Toronto West                                                     20   
Parkdale,Roncesvalles                                                  15   
Rosedale                                                                4   
Roselawn                                                                1   
Runnymede,Swansea                                                      40   
Ryerson,Garden District                                               100   
St. James Town                                                        100   
Stn A PO Boxes 25 The Esplanade                                        95   
Studio District                                                        37   
The Annex,North Midtown,Yorkville                                      24   
The Beaches                                                             4   
The Beaches West,India Bazaar                                          22   
The Danforth West,Riverdale                                            44   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Adelaide,King,Richmond                                                 100   
Berczy Park                                                             57   
Brockton,Exhibition Place,Parkdale Village                              19   
Business Reply Mail Processing Centre 969 Eastern                       34   
CN Tower,Bathurst Quay,Island airport,Harbourfr...                      15   
Cabbagetown,St. James Town                                              44   
Central Bay Street                                                      88   
Chinatown,Grange Park,Kensington Market                                100   
Christie                                                                16   
Church and Wellesley                                                    88   
Commerce Court,Victoria Hotel      

In [127]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


now we analyze each nieghbourhood

In [128]:
# one hot encoding
toronoto_ohe = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronoto_ohe['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronoto_ohe.columns[-1]] + list(toronoto_ohe.columns[:-1])
toronoto_ohe = toronoto_ohe[fixed_columns]

toronoto_ohe.head()

Yoga Studio  Adult Boutique  Afghan Restaurant  Airport  \
0            0               0                  0        0   
1            0               0                  0        0   
2            0               0                  0        0   
3            0               0                  0        0   
4            0               0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  ...  Theme Restaurant  \
0                 0                    0  ...                 0   
1                 0                    0  ...                 0   
2                 0                    0  ...                 0   
3                 0                    0  ...                 0   
4                 0                    0  ...                 0   

   Thrift / Vintage Store  Toy / Game Store  Trail  Train Station  \
0                       0                 0      0              0   
1                       0                 0      0              0   
2                       0                 0      0              0   
3                       0                 0      0              0   
4                       0                 0      0              0   

   Vegetarian / Vegan Restaurant  Video Game Store  Vietnamese Restaurant  \
0                              0                 0                      0   
1                              0                 0                      0   
2                              0                 0                      0   
3                              0                 0                      0   
4                              0                 0                      0   

   Wine Bar  Wings Joint  
0         0            0  
1         0            0  
2         0            0  
3         0            0  
4         0            0  

[5 rows x 236 columns]

In [131]:
toronto_grouped = toronoto_ohe.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

Neighborhood  Yoga Studio  \
0                             Adelaide,King,Richmond     0.000000   
1                                        Berczy Park     0.000000   
2         Brockton,Exhibition Place,Parkdale Village     0.000000   
3  Business Reply Mail Processing Centre 969 Eastern     0.058824   
4  CN Tower,Bathurst Quay,Island airport,Harbourf...     0.000000   

   Adult Boutique  Afghan Restaurant   Airport  Airport Food Court  \
0             0.0                0.0  0.000000            0.000000   
1             0.0                0.0  0.000000            0.000000   
2             0.0                0.0  0.000000            0.000000   
3             0.0                0.0  0.000000            0.000000   
4             0.0                0.0  0.066667            0.066667   

   Airport Gate  Airport Lounge  Airport Service  Airport Terminal  ...  \
0      0.000000        0.000000              0.0          0.000000  ...   
1      0.000000        0.000000              0.0          0.000000  ...   
2      0.000000        0.000000              0.0          0.000000  ...   
3      0.000000        0.000000              0.0          0.000000  ...   
4      0.066667        0.133333              0.2          0.133333  ...   

   Theme Restaurant  Thrift / Vintage Store  Toy / Game Store  Trail  \
0               0.0                     0.0               0.0    0.0   
1               0.0                     0.0               0.0    0.0   
2               0.0                     0.0               0.0    0.0   
3               0.0                     0.0               0.0    0.0   
4               0.0                     0.0               0.0    0.0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0            0.0                       0.010000               0.0   
1            0.0                       0.017544               0.0   
2            0.0                       0.000000               0.0   
3            0.0                       0.000000               0.0   
4            0.0                       0.000000               0.0   

   Vietnamese Restaurant  Wine Bar  Wings Joint  
0                    0.0      0.01          0.0  
1                    0.0      0.00          0.0  
2                    0.0      0.00          0.0  
3                    0.0      0.00          0.0  
4                    0.0      0.00          0.0  

[5 rows x 236 columns]

#### Let's print each neighborhood along with the top 5 most common venues

In [133]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2          Restaurant  0.04
3  Seafood Restaurant  0.04
4              Bakery  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0  Breakfast Spot  0.11
1            Café  0.11
2     Coffee Shop  0.11
3   Burrito Place  0.05
4             Bar  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Garden Center  0.06
3          Comic Shop  0.06
4                Park  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.20
1    Airport Loun

In [136]:
import numpy as np

In [134]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [137]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                             Adelaide,King,Richmond           Coffee Shop   
1                                        Berczy Park           Coffee Shop   
2         Brockton,Exhibition Place,Parkdale Village                  Café   
3  Business Reply Mail Processing Centre 969 Eastern    Light Rail Station   
4  CN Tower,Bathurst Quay,Island airport,Harbourf...       Airport Service   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                  Café       Thai Restaurant            Steakhouse   
1          Cocktail Bar                  Café           Cheese Shop   
2        Breakfast Spot           Coffee Shop         Grocery Store   
3                Garden      Recording Studio         Auto Workshop   
4        Airport Lounge      Airport Terminal       Harbor / Marina   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0   American Restaurant                   Bar          Burger Joint   
1        Farmers Market              Beer Bar            Steakhouse   
2    Italian Restaurant  Caribbean Restaurant               Stadium   
3            Skate Park         Burrito Place  Fast Food Restaurant   
4      Sculpture Garden              Boutique                 Plane   

  8th Most Common Venue   9th Most Common Venue 10th Most Common Venue  
0                   Gym                   Hotel                 Bakery  
1    Seafood Restaurant              Restaurant                 Bakery  
2                   Bar  Furniture / Home Store          Burrito Place  
3        Farmers Market                 Brewery             Restaurant  
4         Boat or Ferry            Airport Gate                Airport

# now let's make the clusters

In [138]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


In [164]:

toronto_merged 


lat     long                                            borough  \
0   43.6543 -79.3606                  Downtown Toronto,Downtown Toronto   
1   43.6572 -79.3789                  Downtown Toronto,Downtown Toronto   
2   43.6515 -79.3754                                   Downtown Toronto   
3   43.6764  -79.293                                       East Toronto   
4   43.6448 -79.3733                                   Downtown Toronto   
5    43.658 -79.3874                                   Downtown Toronto   
6   43.6695 -79.4226                                   Downtown Toronto   
7   43.6506 -79.3846  Downtown Toronto,Downtown Toronto,Downtown Tor...   
8    43.669 -79.4423                          West Toronto,West Toronto   
9   43.6408 -79.3818  Downtown Toronto,Downtown Toronto,Downtown Tor...   
10  43.6479 -79.4197                          West Toronto,West Toronto   
11  43.6796 -79.3522                          East Toronto,East Toronto   
12  43.6472 -79.3816                  Downtown Toronto,Downtown Toronto   
13  43.6368 -79.4282             West Toronto,West Toronto,West Toronto   
14   43.669 -79.3156                          East Toronto,East Toronto   
15  43.6482 -79.3798                  Downtown Toronto,Downtown Toronto   
16  43.6595 -79.3409                                       East Toronto   
17   43.728 -79.3888                                    Central Toronto   
18  43.7117 -79.4169                                    Central Toronto   
19  43.7128 -79.3902                                    Central Toronto   
20  43.6969 -79.4113                    Central Toronto,Central Toronto   
21  43.6616 -79.4648                          West Toronto,West Toronto   
22  43.7154 -79.4057                                    Central Toronto   
23  43.6727 -79.4057    Central Toronto,Central Toronto,Central Toronto   
24   43.649 -79.4563                          West Toronto,West Toronto   
25  43.7043 -79.3888                                    Central Toronto   
26  43.6627    -79.4                  Downtown Toronto,Downtown Toronto   
27  43.6516 -79.4844                          West Toronto,West Toronto   
28  43.6896 -79.3832                    Central Toronto,Central Toronto   
29  43.6532    -79.4  Downtown Toronto,Downtown Toronto,Downtown Tor...   
30  43.6864    -79.4  Central Toronto,Central Toronto,Central Toront...   
31  43.6289 -79.3944  Downtown Toronto,Downtown Toronto,Downtown Tor...   
32  43.6796 -79.3775                                   Downtown Toronto   
33  43.6464 -79.3748                                   Downtown Toronto   
34   43.668 -79.3677                  Downtown Toronto,Downtown Toronto   
35  43.6484 -79.3823                  Downtown Toronto,Downtown Toronto   
36  43.6659 -79.3832                                   Downtown Toronto   
37  43.6627 -79.3216                                       East Toronto   
38  43.6627 -79.3216                                       East Toronto   

                                         Neighborhood  Cluster Labels  \
0                            Harbourfront,Regent Park               1   
1                             Ryerson,Garden District               1   
2                                      St. James Town               1   
3                                         The Beaches               1   
4                                         Berczy Park               1   
5                                  Central Bay Street               1   
6                                            Christie               1   
7                              Adelaide,King,Richmond               1   
8                         Dovercourt Village,Dufferin               1   
9     Harbourfront East,Toronto Islands,Union Station               1   
10                            Little Portugal,Trinity               1   
11                        The Danforth West,Riverdale               1   
12            Design Exchange,Toronto Dominion Centre     

In [166]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['lat'], toronto_merged['long'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters